In [ ]:
!pip install requests_cache


In [ ]:
import socket
from cryptography.fernet import Fernet
import requests_cache
import itertools
import requests

requests_cache.install_cache('demo_cache')

servers = ['http://localhost:8080', 'http://localhost:8081', 'http://localhost:8082']
server_cycle = itertools.cycle(servers)

encryption_key = Fernet.generate_key()
encryption_suite = Fernet(encryption_key)


def send_message(client_socket, message):
    client_socket.sendall(message.encode())


def process_file_request(client_socket):
    file_name = client_socket.recv(1024).decode()

    server_address = next(server_cycle)
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.connect((server_address, 8080))

    server_socket.sendall(file_name.encode())

    file_data = server_socket.recv(1024)
    while file_data:
        encrypted_data = encryption_suite.encrypt(file_data)
        client_socket.sendall(encrypted_data)
        file_data = server_socket.recv(1024)

    server_socket.close()
    client_socket.close()


def validate_credentials(client_socket):
    username = client_socket.recv(1024).decode()
    password = client_socket.recv(1024).decode()

    if username == "admin" and password == "password":
        send_message(client_socket, "Authentication successful")
        return True
    else:
        send_message(client_socket, "Authentication failed")
        return False


def start_proxy_server():
    proxy_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    proxy_socket.bind(('localhost', 8888))
    proxy_socket.listen(5)

    print("Server is up and running. Proxy functionality is active.")


    while True:
        client_socket, client_address = proxy_socket.accept()
        print("Client connected:", client_address)

        if validate_credentials(client_socket):
            process_file_request(client_socket)

start_proxy_server()